In [ ]:
import requests
# response = requests.get('https://pythonexamples.org/python-basic-examples/')
response = requests.get('https://api.gemini.com/v1/book/btcusd')
print(response.json())

In [2]:
import requests

def get_orderbook():
    orderbook = requests.get("https://api.gemini.com/v1/book/btcusd").json()

In [3]:
import requests
import timeit

def get_orderbook():
    orderbook = requests.get("https://api.gemini.com/v1/book/btcusd").json()

n = 10
latency = timeit.timeit('get_orderbook()', setup='from __main__ import get_orderbook', number=n) * 1.0 / n
print('Latency is {} ms'.format(latency * 1000))

Latency is 2960.6118452000032 ms


In [6]:
import time
import websocket
import _thread

def on_message(ws, message):
    print('Received: ' + message)

# 在接收到服务器发送消息时调用
def on_open(ws):
    # 线程运行函数
    def gao():
        # 往服务器依次发送0-4，每次发送完休息0.01秒
        for i in range(5):
            time.sleep(0.01)
            msg = "{0}".format(i)
            ws.send(msg)
            print('Sent: ' + msg)
        # 休息1秒钟用于接受服务器回复的消息
        time.sleep(1)

        # 关闭Websocket的连接
        ws.close()
        print("Websocket closed")
    
    # 在另一个线程运行gao()函数
    _thread.start_new_thread(gao, ())

if __name__ == "__main__":
    ws = websocket.WebSocketApp("ws://echo.websocket.org/",
    on_message = on_message, 
    on_open = on_open)

    ws.run_forever()

ERROR:websocket:Handshake status 200 OK - goodbye


In [3]:
import ssl
import websocket
import json

# 全局计数器
count = 5


def on_message(ws, message):
    global count
    print(message)
    count -= 1
    # 接收了5次消息之后关闭websocket连接
    if count == 0:
        ws.close()


if __name__ == "__main__":
    ws = websocket.WebSocketApp(
        "wss://api.gemini.com/v1/marketdata/btcusd?top_of_book=true&offers=true",
        on_message=on_message)
    ws.run_forever(sslopt={"cert_reqs": ssl.CERT_NONE})


{"type":"update","eventId":162877043088,"socket_sequence":0,"events":[{"type":"change","reason":"initial","price":"21682.01","delta":"0.03400341","remaining":"0.03400341","side":"bid"},{"type":"change","reason":"initial","price":"21691.41","delta":"0.015009","remaining":"0.015009","side":"ask"}]}
{"type":"update","eventId":162877043246,"timestamp":1676110896,"timestampms":1676110896512,"socket_sequence":1,"events":[{"type":"change","side":"bid","price":"21682.02","remaining":"0.05","reason":"top-of-book"}]}
{"type":"update","eventId":162877043250,"timestamp":1676110896,"timestampms":1676110896513,"socket_sequence":2,"events":[{"type":"change","side":"bid","price":"21682.03","remaining":"0.03400341","reason":"top-of-book"}]}
{"type":"update","eventId":162877043394,"timestamp":1676110896,"timestampms":1676110896772,"socket_sequence":3,"events":[{"type":"change","side":"bid","price":"21682.04","remaining":"0.05","reason":"top-of-book"}]}
{"type":"update","eventId":162877043404,"timestamp"

In [ ]:
import copy
import json
import ssl
import time
import websocket


class OrderBook(object):

    BIDS = 'bid'
    ASKS = 'ask'

    def __init__(self, limit=20) -> None:

        self.limit = limit

        # (price, amount)
        self.bids = {}
        self.asks = {}

        self.bids_sorted = []
        self.asks_sorted = []

    def insert(self, price, amount, direction):
        if direction == self.BIDS:
            if amount == 0:
                if price in self.bids:
                    del self.bids[price]
            else:
                self.bids[price] = amount
        elif direction == self.ASKS:
            if amount == 0:
                if price in self.asks:
                    del self.asks[price]
            else:
                self.asks[price] = amount
        else:
            print('WARNING: unknown direction {}'.format(direction))

    def sort_and_truncate(self):
        # sort
        self.bids_sorted = sorted(
            [(price, amount) for price, amount in self.bids.items()], reverse=True)
        self.asks_sorted = sorted([(price, amount)
                                  for price, amount in self.asks.items()])

        # truncate
        self.bids_sorted = self.bids_sorted[:self.limit]
        self.asks_sorted = self.asks_sorted[:self.limit]

        # copy back to bids and asks
        self.bids = dict(self.bids_sorted)
        self.asks = dict(self.asks_sorted)

    def get_copy_of_bids_and_asks(self):
        return copy.deepcopy(self.bids_sorted), copy.deepcopy(self.asks_sorted)


class Crawler:
    def __init__(self, symbol, output_file) -> None:
        self.orderbook = OrderBook(limit=10)
        self.output_file = output_file

        # self.ws = websocket.WebSocketApp('wss://api.gemini.com/v1/marketdata/{}'.format(symbol),
        #                                  on_message=self.on_message)
        self.ws = websocket.WebSocketApp('wss://api.gemini.com/v1/marketdata/{}'.format(symbol),
                                         on_message=lambda ws, message: self.on_message(message))
        self.ws.run_forever(sslopt={'cert_reqs': ssl.CERT_NONE})

    def on_message(self, message):
        # 对收到的信进行处理，然后发送给 orderbook
        data = json.loads(message)
        for event in data['events']:
            price, amount, direction = float(event['price']), float(
                event['remaining']), event['side']
            self.orderbook.insert(price, amount, direction)

        # 整理 orderbook，排序，只选取我们需要的前几个
        self.orderbook.sort_and_truncate()

        # 输出到文件
        with open(self.output_file, 'a+') as f:
            bids, asks = self.orderbook.get_copy_of_bids_and_asks()
            output = {
                'bids': bids,
                'asks': asks,
                'ts': int(time.time() * 1000)
            }
            f.write(json.dumps(output) + '\n')


if __name__ == '__main__':
    crawler = Crawler(symbol='BTCUSD', output_file='BTCUSD.txt')


In [1]:
print(__name__)

__main__
